In [1]:
import os.path as op, os
import nibabel as nib
import numpy as np
import pandas as pd

from copy import deepcopy

from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_mse as mse
from skimage.measure import compare_nrmse as nrmse
from scipy.stats import rice, norm

In [2]:
def oneVoxel(data, loc, scale):
    loc = tuple(loc)
    datnew = np.copy(data)
    if len(loc) < len(data.shape):
        datnew[loc, :] = datnew[loc, :] * (scale)
    else:
        datnew[loc] = datnew[loc] * (scale)
    return datnew

def addNoise(imagestruct, fn, params, noisetype=None):
    noisyim = deepcopy(imagestruct)
    params = [noisyim[-1]] + params  # Assumes first arg to fn is data
    if not noisetype:
        noisetype = str(fn)
    noisyim[4] = "fn: {0}; params: {1}".format(noisetype, str(params[1:]))
    noisyim[6] = fn(*params)
    return noisyim

def rician(data, b, scale):
    datnew = np.copy(data)
    datnew += scale * rice.rvs(b, size=datnew.shape)
    return datnew

def gaussian(data, scale):
    datnew = np.copy(data)
    datnew += scale * norm.rvs(size=datnew.shape)
    datnew[datnew < 0] = 0
    return datnew

In [3]:
diff_df = pd.DataFrame(columns = ["Images", "SameSubject", "SameSession",
                                  "1-voxel", "Rician", "Gaussian", "MSE",
                                  "NRMSE", "SSIM"])
image_df = pd.DataFrame(columns = ["Dataset", "Subject", "Session", "Modality",
                                   "Noise", "Affine", "Image"])

In [4]:
f_image1 = '/data/ds114/sub-03/ses-retest/anat/sub-03_ses-retest_T1w.nii.gz'
f_image2 = '/data/ds114/sub-03/ses-test/anat/sub-03_ses-test_T1w.nii.gz'
f_image3 = '/data/ds114/sub-04/ses-retest/anat/sub-04_ses-retest_T1w.nii.gz'
f_image4 = '/data/ds114/sub-04/ses-test/anat/sub-04_ses-test_T1w.nii.gz'

im1 = nib.load(f_image1)
im2 = nib.load(f_image2)
im3 = nib.load(f_image3)
im4 = nib.load(f_image4)

dat1 = im1.get_data()
dat2 = im2.get_data()
dat3 = im3.get_data()
dat4 = im4.get_data()

image_df.loc[len(image_df)] = ["DS114", "03", "retest", "T1w", None, im1.affine, dat1]
image_df.loc[len(image_df)] = ["DS114", "03",   "test", "T1w", None, im2.affine, dat2]
image_df.loc[len(image_df)] = ["DS114", "04", "retest", "T1w", None, im3.affine, dat3]
image_df.loc[len(image_df)] = ["DS114", "04",   "test", "T1w", None, im4.affine, dat4]

In [5]:
loc = (150, 80, 160)
scale = 1.5
params = [loc, scale]

image_df.loc[len(image_df)] = addNoise(image_df.loc[0][:], oneVoxel, params, noisetype="oneVoxel")
image_df.loc[len(image_df)] = addNoise(image_df.loc[1][:], oneVoxel, params, noisetype="oneVoxel")
image_df.loc[len(image_df)] = addNoise(image_df.loc[2][:], oneVoxel, params, noisetype="oneVoxel")
image_df.loc[len(image_df)] = addNoise(image_df.loc[3][:], oneVoxel, params, noisetype="oneVoxel")

b = 1.1
scale = 1.2
params = [b, scale]

image_df.loc[len(image_df)] = addNoise(image_df.loc[0][:], rician, params, noisetype="Rician")
image_df.loc[len(image_df)] = addNoise(image_df.loc[1][:], rician, params, noisetype="Rician")
image_df.loc[len(image_df)] = addNoise(image_df.loc[2][:], rician, params, noisetype="Rician")
image_df.loc[len(image_df)] = addNoise(image_df.loc[3][:], rician, params, noisetype="Rician")

scale = 1.2
params = [scale]

image_df.loc[len(image_df)] = addNoise(image_df.loc[0][:], gaussian, params, noisetype="Gaussian")
image_df.loc[len(image_df)] = addNoise(image_df.loc[1][:], gaussian, params, noisetype="Gaussian")
image_df.loc[len(image_df)] = addNoise(image_df.loc[2][:], gaussian, params, noisetype="Gaussian")
image_df.loc[len(image_df)] = addNoise(image_df.loc[3][:], gaussian, params, noisetype="Gaussian")

In [6]:
image_df[image_df.columns[:-2]]

,Dataset,Subject,Session,Modality,Noise
0,DS114,03,retest,T1w,None
1,DS114,03,test,T1w,None
2,DS114,04,retest,T1w,None
3,DS114,04,test,T1w,None
4,DS114,03,retest,T1w,"fn: oneVoxel; params: [(150, 80, 160), 1.5]"
5,DS114,03,test,T1w,"fn: oneVoxel; params: [(150, 80, 160), 1.5]"
6,DS114,04,retest,T1w,"fn: oneVoxel; params: [(150, 80, 160), 1.5]"
7,DS114,04,test,T1w,"fn: oneVoxel; params: [(150, 80, 160), 1.5]"
8,DS114,03,retest,T1w,"fn: Rician; params: [1.1, 1.2]"
9,DS114,03,test,T1w,"fn: Rician; params: [1.1, 1.2]"


In [7]:
for idx, im1 in image_df.iterrows():
    for jdx, im2 in image_df.iterrows():
        if jdx <= idx:
            continue
        print((idx, jdx), end=" ")
        diff_df.loc[len(diff_df)] = [(idx, jdx),  # Image indices
                                     im1[1] == im2[1],  # Same subject?
                                     im1[2] == im2[2],  # Same session?
                                     any("oneVoxel" in tim
                                         for tim in [im1[4], im2[4]]
                                         if tim is not None),  # 1-voxel noise?
                                     any("Rician" in tim
                                         for tim in [im1[4], im2[4]]
                                         if tim is not None),  # Rician noise?
                                     any("Gaussian" in tim
                                         for tim in [im1[4], im2[4]]
                                         if tim is not None),  # Gaussian noise?
                                     mse(im1[-1], im2[-1]),
                                     nrmse(im1[-1], im2[-1]),
                                     ssim(im1[-1], im2[-1])]

(0, 1) (0, 2) (0, 3) (0, 4) (0, 5) (0, 6) (0, 7) (0, 8) (0, 9) (0, 10) (0, 11) (0, 12) (0, 13) (0, 14) (0, 15) (1, 2) (1, 3) (1, 4) (1, 5) (1, 6) (1, 7) (1, 8) (1, 9) (1, 10) (1, 11) (1, 12) (1, 13) (1, 14) (1, 15) (2, 3) (2, 4) (2, 5) (2, 6) (2, 7) (2, 8) (2, 9) (2, 10) (2, 11) (2, 12) (2, 13) (2, 14) (2, 15) (3, 4) (3, 5) (3, 6) (3, 7) (3, 8) (3, 9) (3, 10) (3, 11) (3, 12) (3, 13) (3, 14) (3, 15) (4, 5) (4, 6) (4, 7) (4, 8) (4, 9) (4, 10) (4, 11) (4, 12) (4, 13) (4, 14) (4, 15) (5, 6) (5, 7) (5, 8) (5, 9) (5, 10) (5, 11) (5, 12) (5, 13) (5, 14) (5, 15) (6, 7) (6, 8) (6, 9) (6, 10) (6, 11) (6, 12) (6, 13) (6, 14) (6, 15) (7, 8) (7, 9) (7, 10) (7, 11) (7, 12) (7, 13) (7, 14) (7, 15) (8, 9) (8, 10) (8, 11) (8, 12) (8, 13) (8, 14) (8, 15) (9, 10) (9, 11) (9, 12) (9, 13) (9, 14) (9, 15) (10, 11) (10, 12) (10, 13) (10, 14) (10, 15) (11, 12) (11, 13) (11, 14) (11, 15) (12, 13) (12, 14) (12, 15) (13, 14) (13, 15) (14, 15) 

In [10]:
# diff_df.loc[diff_df.SameSubject.values == True].sort_values('SSIM')
diff_df.sort_values('SSIM')

,Images,SameSubject,SameSession,1-voxel,Rician,Gaussian,MSE,NRMSE,SSIM
24,"(1, 11)",False,True,False,True,False,100295.308814,1.078196,0.010815
70,"(5, 11)",False,True,True,True,False,100295.319301,1.078196,0.010815
28,"(1, 15)",False,True,False,False,True,100118.657492,1.077246,0.010819
74,"(5, 15)",False,True,True,False,True,100118.668103,1.077246,0.010819
51,"(3, 13)",False,True,False,False,True,100106.609205,0.897509,0.010835
89,"(7, 13)",False,True,True,False,True,100106.615247,0.897509,0.010835
47,"(3, 9)",False,True,False,True,False,99955.757535,0.896833,0.010836
85,"(7, 9)",False,True,True,True,False,99955.763471,0.896833,0.010836
104,"(9, 15)",False,True,False,True,True,99952.830191,1.073352,0.011236
111,"(11, 13)",False,True,False,True,True,100280.167536,0.895936,0.011314
